In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas 
import ipywidgets
import mne
import os
from mne import Epochs, compute_covariance, find_events, make_ad_hoc_cov
from mne.datasets import sample
from mne.preprocessing import annotate_movement, compute_average_dev_head_t, annotate_muscle_zscore
from mne.datasets.brainstorm import bst_auditory
from mne.io import read_raw_ctf, read_raw_edf
from mne.viz import plot_alignment, set_3d_view
from mne.simulation import (
    add_ecg,
    add_eog,
    add_noise,
    simulate_raw,
    simulate_sparse_stc,
)


EDFInputPath = 'EDFFiles'
EDFOutputPath = 'OutputFiles'


def AllEDFProcess(EDFFolder):
    if not os.path.exists(EDFOutputPath):
        os.makedirs(EDFOutputPath)
    
    for FileName in os.listdir(EDFFolder):
        if FileName.endswith('.edf'):
            EDFFilePath = os.path.join(EDFFolder, FileName)
            EDFProcess(EDFFilePath)
            

def EDFProcess(EDFFilePath):
    RawEEGDataFile = mne.io.read_raw_edf(EDFFilePath, preload=True)
    RawEEGDataFile.interpolate_bads();

    BPEEGDataFile = BPFilter(RawEEGDataFile)

    OutputFileName = f"filtered_{os.path.splitext(os.path.basename(EDFFilePath))[0]}.fif"
    OutputFile = os.path.join(EDFOutputPath, OutputFileName)
    BPEEGDataFile.save(OutputFile, overwrite=True)
    RawEEGDataFile.plot()
    BPEEGDataFile.plot()
    print(f"Finished and saved file {EDFFilePath} to {OutputFile}")

def BPFilter(RawEEGDataFile):
    BPEEGDataFile = RawEEGDataFile.copy().filter(l_freq=0.5, h_freq=40.0, fir_design='firwin')
    return BPEEGDataFile


AllEDFProcess(EDFInputPath)

# RawEEGDataFile = mne.io.read_raw_edf("aaaaaaaa_s001_t000.edf", preload=True)
# raw_data = RawEEGDataFile.get_data()
# info = RawEEGDataFile.info
# channels = RawEEGDataFile.ch_names

# RawEEGDataFile = RawEEGDataFile.pick(picks=["eeg", "eog", "ecg", "stim"]).load_data();
# RawEEGDataFile.interpolate_bads();
# RawEEGDataFile.get_data().shape


#display(raw_data)
#display(info)

# data_path = sample.data_path()
# subjects_dir = data_path / "sample"
# meg_path = data_path / "MEG" / "sample"
# raw_fname = meg_path / "sample_audvis_raw.fif"
# fwd_fname = meg_path / "sample_audvis-meg-eeg-oct-6-fwd.fif"

# Load real data as the template
#RawEEGDataFile = mne.io.read_raw_fif(raw_fname, preload=True);
#RawEEGDataFile.set_eeg_reference(projection=True);
# RawEEGDataFile = RawEEGDataFile.pick(picks=["eeg", "eog", "ecg", "stim"]).load_data();
# RawEEGDataFile.interpolate_bads();
# RawEEGDataFile.get_data().shape
# EEGChannelNum = input("Please input the number of channels:")
#Freq = float(input("Please input sample refresh rate in Hz:"))
# RawEEGDataFile = input("Please input the EEG data file name:")

Freq = 256
EEGChannelNum = 31
ChannelArr = np.arange(1, EEGChannelNum + 1)

#raw_csd.plot();

#raw.compute_psd().plot(picks="data", exclude="bads", amplitude=False);

#raw_csd.compute_psd().plot(picks="data", exclude="bads", amplitude=False);


In [ ]:
# ## BANDPASS FILTERING OF RAW DATA ##

# BPEEGDataFile = RawEEGDataFile.copy().filter(l_freq=0.5, h_freq=40.0, fir_design='firwin')
# #RawEEGDataFile.notch_filter([60, 120])

# RawEEGDataFile.plot()
# BPEEGDataFile.plot()

# RawDF = RawEEGDataFile.to_data_frame()
# display(RawDF)
# BPDF = BPEEGDataFile.to_data_frame()
# display(BPDF)

In [ ]:
## ALPHA DELTA PSD ANALYSIS AND DATA FRAMING ##
def AlphaDeltaProcess(EEGFile):
    AlphaComp = EEGFile.compute_psd(method='welch', fmin=8, fmax=12, tmin=None, tmax=None, picks='eeg', exclude=(), proj=False, remove_dc=True, reject_by_annotation=True, n_jobs=1, verbose=None)
    AlphaPSD, AlphaFreq = AlphaComp.get_data(return_freqs=True)
    #display(AlphaComp)
    DeltaComp = EEGFile.compute_psd(method='welch', fmin=0.5, fmax=4, tmin=None, tmax=None, picks='eeg', exclude=(), proj=False, remove_dc=True, reject_by_annotation=True, n_jobs=1, verbose=None)
    DeltaPSD, DeltaFreq = DeltaComp.get_data(return_freqs=True)
    #DeltaComp.plot()
    #raw_csd = mne.preprocessing.compute_current_source_density(RawEEGDataFile);

    AlphaMean = AlphaPSD.mean(axis=1)
    DeltaMean = DeltaPSD.mean(axis=1)

    AlDeRat = AlphaMean / DeltaMean

    PSDRatDF = pandas.DataFrame({'Channel': ChannelArr,'Alpha Power': AlphaMean,'Delta Power': DeltaMean,'Alpha/Delta Ratio': AlDeRat})

    #display(PSDRatDF)
    return PSDRatDF

In [ ]:
from mne.viz import plot_alignment, set_3d_view
trans = mne.read_trans(meg_path / 'sample_audvis_raw-trans.fif')
#raw = mne.io.read_raw_fif(meg_path / 'sample_audvis_raw.fif')

# Plot electrode locations on scalp
fig = plot_alignment(RawEEGDataFile.info, trans, subject='sample', dig=False,
                     eeg=['original', 'projected'], meg=[],
                     coord_frame='head', subjects_dir=subjects_dir)

# Set viewing angle
set_3d_view(figure=fig, azimuth=135, elevation=80)